<a href="https://colab.research.google.com/github/gerritgr/icon/blob/main/icon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 icon: Fast Simulation of Epidemics on Coevolving Networks

### Setup

In [29]:
# Import necessary libraries
import networkx as nx
import numpy as np
import random
from itertools import combinations
from scipy.stats import expon
import pandas as pd
import time
from tqdm import tqdm
import copy
import itertools
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# Simulation time limit set to 120 minutes
SIM_TIME_LIMIT_IN_S = 60 * 120

### Utils

In [23]:
# Function to compute the epidemic threshold, too slow for very large graphs :(
# Thus, we use  the average degree as a simple proxy
def compute_epidemic_threshold_or_avg_degree(G, fast_approx=True):
    if fast_approx:
        avg_degree = 2 * G.number_of_edges() / G.number_of_nodes()
        return 1 / avg_degree

    A = nx.adjacency_matrix(G).todense()
    eigenvalues = np.linalg.eigvals(A)
    largest_eigenvalue = max(eigenvalues.real)
    return 1 / largest_eigenvalue


# Create a random geometric graph based on target degeee
def create_geometric_graph(n, target_degree=10, seed=42):
    r = np.sqrt(target_degree / (n * np.pi))
    G = nx.random_geometric_graph(n, r, seed=seed)
    avg_degree = sum(dict(G.degree()).values()) / n
    return G


### Baseline (naive)

In [24]:
# Function to run the baseline epidemic simulation
def run_baseline_simulation(
        G, node_to_state_map, filename, beta_rate, alpha_rate,
        b_rate, a_rate,
        horizon, save_graph_state, graph_type):

    # Ensure all nodes in node_to_state_map are added to the graph
    for node in node_to_state_map:
        if node not in G.nodes():
            G.add_node(node)

    n = len(G.nodes())  # Number of nodes in the graph

    # Initial count of infected nodes
    infected_nodes_count = sum(1 for state in node_to_state_map.values() if state == 'I')

    # Milestones to record simulation progress (we store the results when milestone is passed)
    milestones = np.linspace(0, horizon, 101)
    milestone_index = 0

    # Lists to store results at each milestone
    infected_nodes_records = []
    avg_degree_records = []
    accepted_events_records = []
    rejected_events_records = []  # Empty since no rejections in baseline
    edge_list_records = [] if save_graph_state else None

    # Initialize accepted event counter (no rejected events happen in this baseline)
    accepted_events = 0

    # Start measuring runtime
    start_time = time.time()

    # Simulation loop with progress bar
    t = 0
    with tqdm(total=horizon, desc=f"Simulating {graph_type} graph with {n} nodes", dynamic_ncols=True) as pbar:
        while t < horizon:
            # Record results at milestones
            while milestone_index < len(milestones) and t >= milestones[milestone_index]:
                infected_nodes_records.append(infected_nodes_count)
                avg_degree_records.append(2 * len(G.edges()) / n)  # Formula do compute avg. degree
                accepted_events_records.append(accepted_events)
                rejected_events_records.append(0)  # No rejections in baseline
                if save_graph_state: # Use ; because we save as .csv
                    edge_list_records.append(';'.join([f"({u},{v})" for u, v in G.edges()]))
                milestone_index += 1

            # Prepare list of possible events and their jump times
            events = []

            # Edge events (transmission or disconnection)
            for v1, v2 in itertools.combinations(G.nodes(), 2):
                if G.has_edge(v1, v2):
                    # SI edge: Transmission
                    if node_to_state_map[v1] != node_to_state_map[v2]:
                        jump_time = expon(scale=1 / beta_rate).rvs()
                        events.append((jump_time, 'transmission', min(v1, v2), max(v1, v2)))
                    # II edge: Disconnection
                    elif node_to_state_map[v1] == 'I' and node_to_state_map[v2] == 'I':
                        jump_time = expon(scale=1 / b_rate).rvs()
                        events.append((jump_time, 'disconnection', min(v1, v2), max(v1, v2)))
                else:
                    # SS nodes: Connection event
                    if node_to_state_map[v1] == 'S' and node_to_state_map[v2] == 'S':
                        jump_time = expon(scale=1 / a_rate).rvs()
                        events.append((jump_time, 'connection', min(v1, v2), max(v1, v2)))

            # Recovery events
            for v in G.nodes():
                if node_to_state_map[v] == 'I':
                    jump_time = expon(scale=1 / alpha_rate).rvs()
                    events.append((jump_time, 'recovery', v, None))

            # Find the event with the smallest jump time
            events.sort(key=lambda x: x[0])
            if events:
                min_time, event_type, v1, v2 = events[0]

            # Apply the event
            t += min_time
            if time.time() - start_time > SIM_TIME_LIMIT_IN_S or len(events) == 0:
                t += horizon* 2 # Enforce termination
            if t >= horizon:
                pbar.n = horizon # Set progress bar to final value H
                pbar.refresh()
                break

            # Update progress bar
            pbar.n = t
            pbar.refresh()

            # Process the event
            if event_type == 'transmission':
                infected_nodes_count += 1
                # Transmission event: one node infects the other
                node_to_state_map[v2] = 'I'
            elif event_type == 'disconnection':
                # Disconnection event: remove the edge
                G.remove_edge(v1, v2)
            elif event_type == 'connection':
                # Connection event: add edge between SS nodes
                G.add_edge(v1, v2)
            elif event_type == 'recovery':
                # Recovery event: node becomes susceptible
                if infected_nodes_count != 1:
                    node_to_state_map[v1] = 'S'
                    infected_nodes_count -= 1

            accepted_events += 1

    # Record remaining milestones if the time horizon is reached
    while milestone_index < len(milestones):
        infected_nodes_records.append(infected_nodes_count)
        avg_degree_records.append(2 * len(G.edges()) / n)
        accepted_events_records.append(accepted_events)
        rejected_events_records.append(0)
        if save_graph_state:
            edge_list_records.append(';'.join([f"({u},{v})" for u, v in G.edges()]))
        milestone_index += 1

    # Measure the runtime in milliseconds
    run_time_in_ms = (time.time() - start_time) * 1000

    # Prepare data for saving
    data = {
        "experiment_id": [experiment_num] * len(milestones),
        "Time": milestones,
        "Avg_degree": avg_degree_records,
        "Infected_nodes": infected_nodes_records,
        "Accepted_events": accepted_events_records,
        "Rejected_events": rejected_events_records,
        "Run_time_in_ms": [run_time_in_ms] * len(milestones),
        "Number_of_nodes": [n] * len(milestones),
        "graph_type": [graph_type] * len(milestones)
    }

    # Save results to CSV
    results_df = pd.DataFrame(data)
    results_df.to_csv(filename, index=False)

    print(f"Baseline simulation for {graph_type} graph with {n} nodes complete. Results saved to {filename}.")


### Baseline (fast)

In [25]:
def run_baseline_simulation_fast(
        G, node_to_state_map, filename, beta_rate, alpha_rate,
        b_rate, a_rate,
        horizon, save_graph_state, graph_type):

    # Add missing nodes from node_to_state_map to the graph
    for node in node_to_state_map:
        if node not in G.nodes():
            G.add_node(node)

    # Initial state: count infected nodes
    infected_nodes_count = sum(1 for state in node_to_state_map.values() if state == 'I')

    # Initialize data structures with ordered node pairs
    SS_nonedge = [
        (min(u, v), max(u, v)) for u in G.nodes() for v in G.nodes()
        if u < v and node_to_state_map[u] == 'S' and node_to_state_map[v] == 'S' and not G.has_edge(u, v)
    ]
    SI_edge = [
        (min(u, v), max(u, v)) for u, v in G.edges()
        if (node_to_state_map[u] == 'S' and node_to_state_map[v] == 'I') or (node_to_state_map[u] == 'I' and node_to_state_map[v] == 'S')
    ]
    I_node = [v for v in G.nodes() if node_to_state_map[v] == 'I']
    II_edge = [
        (min(u, v), max(u, v)) for u, v in G.edges() if node_to_state_map[u] == 'I' and node_to_state_map[v] == 'I'
    ]

    # Ensure consistency of the initial data structures
    for u, v in SI_edge:
        assert node_to_state_map[u] != node_to_state_map[v], "Initial inconsistent SI_edge list"
    for u, v in II_edge:
        assert node_to_state_map[u] == 'I' and node_to_state_map[v] == 'I', "Initial inconsistent II_edge list"
    for u, v in SS_nonedge:
        assert node_to_state_map[u] == 'S' and node_to_state_map[v] == 'S' and not G.has_edge(u, v), "Initial inconsistent SS_nonedge list"

    # Milestones
    milestones = np.linspace(0, horizon, 101)
    milestone_index = 0

    # Lists to store results at milestones
    infected_nodes_records = []
    avg_degree_records = []
    accepted_events_records = []
    rejected_events_records = []  # No rejections in baseline simulation
    edge_list_records = [] if save_graph_state else None

    # Initialize counter for accepted events
    accepted_events = 0

    # Start measuring the runtime
    start_time = time.time()

    # Simulation loop with progress bar
    t = 0
    n = len(G.nodes())  # Total number of nodes
    with tqdm(total=horizon, desc=f"Simulating {graph_type} graph with {n} nodes", dynamic_ncols=True) as pbar:
        while t < horizon:
            # Record results at milestones
            while milestone_index < len(milestones) and t >= milestones[milestone_index]:
                infected_nodes_records.append(infected_nodes_count)
                avg_degree_records.append(2 * len(G.edges()) / n)  # Average degree
                accepted_events_records.append(accepted_events)
                rejected_events_records.append(0)  # No rejections in baseline
                if save_graph_state:
                    edge_list_records.append(';'.join([f"({u},{v})" for u, v in G.edges()]))
                milestone_index += 1

            # Calculate jump times for each event type
            time_to_transmission = np.random.exponential(1 / (len(SI_edge) * beta_rate)) if SI_edge else float('inf')
            time_to_disconnection = np.random.exponential(1 / (len(II_edge) * b_rate)) if II_edge else float('inf')
            time_to_connection = np.random.exponential(1 / (len(SS_nonedge) * a_rate)) if SS_nonedge else float('inf')
            time_to_recovery = np.random.exponential(1 / (len(I_node) * alpha_rate)) if I_node else float('inf')

            # Determine the event with the shortest jump time
            min_time = min(time_to_transmission, time_to_disconnection, time_to_connection, time_to_recovery)

            # Apply the event with the shortest jump time
            t += min_time
            if time.time() - start_time > SIM_TIME_LIMIT_IN_S:
                t += horizon* 2  # Stop simulation if it exceeds time limit
            if t >= horizon:
                pbar.n = horizon # Set progress bar to final value H
                pbar.refresh()
                break

            # Update progress bar
            pbar.n = t
            pbar.refresh()

            # Handle the events based on the minimum jump time
            if min_time == time_to_transmission:
                # Transmission: Choose a random SI edge
                u, v = random.choice(SI_edge)
                new_infected = u if node_to_state_map[u] == 'S' else v
                node_to_state_map[new_infected] = 'I'
                infected_nodes_count += 1
                I_node.append(new_infected)

                # Update II edges with the newly infected node
                for neighbor in G.neighbors(new_infected):
                    if node_to_state_map[neighbor] == 'I':
                        II_edge.append((min(new_infected, neighbor), max(new_infected, neighbor)))

                # Update SS_nonedge and SI_edge lists
                SS_nonedge = [(x, y) for x, y in SS_nonedge if node_to_state_map[x] == 'S' and node_to_state_map[y] == 'S']
                SI_edge = [
                    (a, b) for a, b in SI_edge
                    if (node_to_state_map[a] == 'I' and node_to_state_map[b] == 'S') or (node_to_state_map[a] == 'S' and node_to_state_map[b] == 'I')
                ]
                for neighbor in G.neighbors(new_infected):
                    if node_to_state_map[neighbor] == 'S':
                        SI_edge.append((min(new_infected, neighbor), max(new_infected, neighbor)))

            elif min_time == time_to_disconnection:
                # Disconnection: Remove a random II edge
                u, v = random.choice(II_edge)
                G.remove_edge(u, v)
                II_edge.remove((u, v))

            elif min_time == time_to_connection:
                # Connection: Add a random SS non-edge
                u, v = random.choice(SS_nonedge)
                G.add_edge(u, v)
                SS_nonedge.remove((u, v))

            elif min_time == time_to_recovery:
                # Recovery: Choose a random infected node to recover
                if infected_nodes_count != 1:
                    v = random.choice(I_node)
                    node_to_state_map[v] = 'S'
                    infected_nodes_count -= 1
                    I_node.remove(v)

                    # Update SI_edge and II_edge
                    for neighbor in G.neighbors(v):
                        if node_to_state_map[neighbor] == 'S':
                            SI_edge.remove((min(v, neighbor), max(v, neighbor)))
                        elif node_to_state_map[neighbor] == 'I':
                            II_edge.remove((min(v, neighbor), max(v, neighbor)))
                            SI_edge.append((min(v, neighbor), max(v, neighbor)))

                    # Update SS_nonedge
                    for other_node in G.nodes():
                        if other_node != v and node_to_state_map[other_node] == 'S' and not G.has_edge(v, other_node):
                            SS_nonedge.append((min(v, other_node), max(v, other_node)))

            accepted_events += 1

    # Ensure remaining milestones are recorded if the time horizon is reached
    while milestone_index < len(milestones):
        infected_nodes_records.append(infected_nodes_count)
        avg_degree_records.append(2 * len(G.edges()) / n)  # Average degree
        accepted_events_records.append(accepted_events)
        rejected_events_records.append(0)  # No rejections in baseline
        if save_graph_state:
            edge_list_records.append(';'.join([f"({u},{v})" for u, v in G.edges()]))
        milestone_index += 1

    # Measure the runtime in milliseconds
    run_time_in_ms = (time.time() - start_time) * 1000

    # Prepare data for saving
    data = {
        "experiment_id": [experiment_num] * len(milestones),
        "Time": milestones,
        "Avg_degree": avg_degree_records,
        "Infected_nodes": infected_nodes_records,
        "Accepted_events": accepted_events_records,
        "Rejected_events": rejected_events_records,  # No rejections in baseline
        "Run_time_in_ms": [run_time_in_ms] * len(milestones),
        "Number_of_nodes": [n] * len(milestones),
        "graph_type": [graph_type] * len(milestones)
    }

    # Save results to CSV
    results_df = pd.DataFrame(data)
    results_df.to_csv(filename, index=False)

    print(f"Fast Baseline simulation for {graph_type} graph with {n} nodes complete. Results saved to {filename}.")


### 🎯 icon: our Method

In [26]:
# Function to run the epidemic simulation
def run_simulation(G, node_to_state_map, filename, beta_rate, alpha_rate,
                   b_rate, a_rate,
                   horizon, save_graph_state, graph_type):

    # Ensure all nodes from node_to_state_map are present in the graph
    for node in node_to_state_map:
        if node not in G.nodes():
            G.add_node(node)

    # Get the total number of nodes
    n = len(G.nodes())

    # Count the initial number of infected nodes
    infected_nodes_count = sum(1 for state in node_to_state_map.values() if state == 'I')

    # Create a unique edge list with (v, v') where v < v'
    edge_list = [(min(u, v), max(u, v)) for u, v in G.edges()]
    edge_list = list(set(edge_list))

    # Calculate the initial average degree
    avg_degree = 2 * len(edge_list) / n

    # Determine upper bounds for various rates
    upper_bound_edge = max(beta_rate, b_rate)
    upper_bound_node = alpha_rate
    upper_bound_possible_pairs = n * (n - 1) // 2
    upper_bound_non_connected = a_rate

    # Milestones for recording results during the simulation
    milestones = np.linspace(0, horizon, 101)
    milestone_index = 0

    # Lists to store results at each milestone
    infected_nodes_records = []
    avg_degree_records = []
    accepted_events_records = []
    rejected_events_records = []
    edge_list_records = [] if save_graph_state else None

    # Counters for accepted and rejected events
    accepted_events = 0
    rejected_events = 0

    # Start measuring the runtime
    start_time = time.time()

    # Simulation loop with progress bar
    t = 0
    with tqdm(total=horizon, desc=f"Simulating {graph_type} graph with {n} nodes", dynamic_ncols=True) as pbar:
        while t < horizon:
            # Record results at milestones
            while milestone_index < len(milestones) and t >= milestones[milestone_index]:
                infected_nodes_records.append(infected_nodes_count)
                avg_degree_records.append(avg_degree)
                accepted_events_records.append(accepted_events)
                rejected_events_records.append(rejected_events)
                if save_graph_state:
                    edge_list_records.append(';'.join([f"({u},{v})" for u, v in edge_list]))
                milestone_index += 1

            # Rates based on the current state of the system
            rate_edge = len(edge_list) * upper_bound_edge
            rate_node = n * upper_bound_node
            rate_non_connected = upper_bound_possible_pairs * upper_bound_non_connected

            # Generate residence times for each type of event
            time_to_edge_event = expon(scale=1/rate_edge).rvs() if rate_edge > 0 else float('inf')
            time_to_node_event = expon(scale=1/rate_node).rvs() if rate_node > 0 else float('inf')
            time_to_non_connected_event = expon(scale=1/rate_non_connected).rvs() if rate_non_connected > 0 else float('inf')

            # Determine which event happens next
            times = [time_to_edge_event, time_to_node_event, time_to_non_connected_event]
            event_index = np.argmin(times)
            t += times[event_index]

            # Stop the simulation if time limit is exceeded or no more infected nodes
            if time.time() - start_time > SIM_TIME_LIMIT_IN_S or infected_nodes_count == 0:
                t += horizon* 2

            if t >= horizon:
                pbar.n = horizon # Set progress bar to the final value H
                pbar.refresh()
                break

            # Update the progress bar
            pbar.n = t
            pbar.refresh()

            # Process edge event (SI transmission or II disconnection)
            if event_index == 0:
                u, v = random.choice(edge_list)
                if node_to_state_map[u] != node_to_state_map[v]:  # SI edge
                    if random.random() < beta_rate / upper_bound_edge:
                        if node_to_state_map[u] == 'I':
                            node_to_state_map[v] = 'I'
                        else:
                            node_to_state_map[u] = 'I'
                        infected_nodes_count += 1
                        accepted_events += 1
                    else:
                        rejected_events += 1
                elif node_to_state_map[u] == 'I' and node_to_state_map[v] == 'I':  # II edge
                    if random.random() < b_rate / upper_bound_edge:
                        edge_list.remove((u, v))
                        avg_degree = 2 * len(edge_list) / n
                        accepted_events += 1
                    else:
                        rejected_events += 1

            # Process node event (recovery)
            elif event_index == 1:
                u = random.choice(list(G.nodes()))
                if node_to_state_map[u] == 'I':
                    if random.random() < alpha_rate/ upper_bound_node:
                        if infected_nodes_count != 1:
                            node_to_state_map[u] = 'S'
                            infected_nodes_count -= 1
                        accepted_events += 1
                    else:
                        rejected_events += 1
                else:
                    rejected_events += 1

            # Process non-connected event (SS connection)
            elif event_index == 2:
                u, v = sorted(random.sample(range(n), 2))
                if (u, v) not in edge_list and node_to_state_map[u] == 'S' and node_to_state_map[v] == 'S':
                    if random.random() < a_rate / upper_bound_non_connected:
                        edge_list.append((u, v))
                        avg_degree = 2 * len(edge_list) / n
                        accepted_events += 1
                    else:
                        rejected_events += 1
                else:
                    rejected_events += 1

    # Ensure remaining milestones are recorded if the time horizon is reached
    while milestone_index < len(milestones):
        infected_nodes_records.append(infected_nodes_count)
        avg_degree_records.append(avg_degree)
        accepted_events_records.append(accepted_events)
        rejected_events_records.append(rejected_events)
        if save_graph_state:
            edge_list_records.append(';'.join([f"({u},{v})" for u, v in edge_list]))
        milestone_index += 1

    # Measure the runtime in milliseconds
    run_time_in_ms = (time.time() - start_time) * 1000

    # Prepare the data for saving
    data = {
        "experiment_id": [experiment_num] * len(milestones),
        "Time": milestones,
        "Avg_degree": avg_degree_records,
        "Infected_nodes": infected_nodes_records,
        "Accepted_events": accepted_events_records,
        "Rejected_events": rejected_events_records,
        "Run_time_in_ms": [run_time_in_ms] * len(milestones),
        "Number_of_nodes": [n] * len(milestones),
        "graph_type": [graph_type] * len(milestones)
    }

    # Save results to CSV
    results_df = pd.DataFrame(data)
    if save_graph_state:
        results_df["edge_list"] = edge_list_records

    results_df.to_csv(filename, index=False)

    print(f"Experiment {experiment_num} complete. Results saved to {filename}.")



### Start Experiments

In [ ]:
# Model parameters
prob_init_infected = 0.1
beta_rate = 2.0 # transmission rate
b_rate = 2.0 # edge disconnection rate
a_star = 0.3 # rate for connection of SS edges
alpha_rate= 1.0 # rate for recovery

beta_func = lambda x: x * beta_rate # maps avg degree to beta
a_rate_function = lambda n: a_star / n # maps number of nodes to a

# Experiment parameters
n_values = [10, 10**2, 10**3, 10**4, 10**5] # number of nodes
experiment_num = 1
save_graph_state_limit = 11  # Limit to save the graph state (edge_list) at each milestone
experiments_per_n = 5
graph_types = ['random', 'geometric', 'barabasi']
random_seed = 42  #


# Function to determine time horizon based on the number of nodes
def H_func(n):
    if n >= 10000:
        return 0.1
    if n >= 1000:
        return 1.0
    return 5.0


# Set the random seed for reproducibility
random.seed(random_seed)
np.random.seed(random_seed)


# Main simulation loop
for n in n_values:
    for graph_type in graph_types:
        for _ in range(experiments_per_n):
            print("Starting experiment...")

            # Generate the graph based on the specified type
            if graph_type == 'random':
                # Generate a connected random graph (Erdős–Rényi). Relax if too expensive.
                for i in range(10):  # Try generating the graph multiple times
                    target_degree = 5  # Average degree is 5
                    G = nx.erdos_renyi_graph(n, target_degree / (n - 1), seed=42+i+experiment_num)
                    if n > 1000 or nx.is_connected(G):
                        break

            elif graph_type == 'barabasi':
                # Generate a Barabási–Albert graph
                G = nx.barabasi_albert_graph(n, 5, seed=42+experiment_num)

            elif graph_type == 'geometric':
                # Generate a geometric graph
                target_degree = 5
                G = create_geometric_graph(n, target_degree=target_degree, seed=42+experiment_num)

            print(f"Graph generated with {G.number_of_nodes()} nodes.")

            # Calculate transmission rate, connection rate, and time horizon for concrete graph
            beta_rate = beta_func(compute_epidemic_threshold_or_avg_degree(G))
            a_rate = a_rate_function(n)
            horizon= H_func(n)

            # Initialize node states (S or I)
            num_infected = int(n * prob_init_infected + 0.5)
            assert num_infected > 0
            infected_nodes = set(random.sample(list(G.nodes()), num_infected))
            node_to_state_map = {node: 'I' if node in infected_nodes else 'S' for node in G.nodes()}

            # Define the filename for saving results
            filename = f'sis_simulation_results_{experiment_num:03d}_{n:010d}_nodes_{graph_type}.csv'

            # Run the main simulation and save results
            print("Running simulation...")
            save_graph_state = n <= save_graph_state_limit
            run_simulation(copy.deepcopy(G), copy.deepcopy(node_to_state_map), filename, beta_rate, alpha_rate,
                           b_rate, a_rate,
                           horizon, save_graph_state, graph_type)

            # Optionally run the baseline fast simulation if n <= 10,000
            try:
                if n <= 10000:
                    print("Running baseline fast simulation...")
                    filename = f'sis_baselinefast_simulation_results_{experiment_num:03d}_{n:010d}_nodes_{graph_type}.csv'
                    run_baseline_simulation_fast(copy.deepcopy(G), copy.deepcopy(node_to_state_map), filename, beta_rate, alpha_rate,
                                                 b_rate, a_rate,
                                                 horizon, save_graph_state, graph_type)
            except Exception as e:
                print(f"Exception during baseline fast simulation: {e}")

            # Optionally run the baseline naive simulation if n <= 1,000
            try:
                if n <= 1000:
                    print("Running baseline naive simulation...")
                    filename = f'sis_baselinenaive_simulation_results_{experiment_num:03d}_{n:010d}_nodes_{graph_type}.csv'
                    run_baseline_simulation(copy.deepcopy(G), copy.deepcopy(node_to_state_map), filename, beta_rate, alpha_rate,
                                            b_rate, a_rate,
                                            horizon, save_graph_state, graph_type)
            except Exception as e:
                print(f"Exception during baseline naive simulation: {e}")

            # Increment experiment number
            experiment_num += 1


### Plotting Trajectories

In [ ]:
# Set seaborn style for publication-quality plots, without grid
sns.set(style="white", rc={"axes.edgecolor": "0.0", "axes.linewidth": 1.25})

# Function to format plots for publication
def format_plot(ax, title, xlabel, ylabel):
    """Format the plot for publication-ready figures."""
    ax.set_title(title, fontsize=25)
    ax.set_xlabel(xlabel, fontsize=30)
    ax.set_ylabel(ylabel, fontsize=30)
    ax.tick_params(axis='both', which='major', labelsize=30, length=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Initialize dictionaries to store data and filenames
data_dict = dict()
filenames = dict()

# Get all files matching the pattern
files = glob.glob("sis_simulation_results_*.csv")

# Process each file and store data in the dictionaries
for file in sorted(files):
    print(file)
    n = int(file.split("_")[4])
    graph_type = file.split("_")[6].replace('.csv', '')

    # Initialize lists if key doesn't exist
    if (n, graph_type) not in data_dict:
        data_dict[(n, graph_type)] = []
    if (n, graph_type) not in filenames:
        filenames[(n, graph_type)] = []

    # Append data and filenames
    data_dict[(n, graph_type)].append(pd.read_csv(file))
    filenames[(n, graph_type)].append(file)

# Define the styles: alternating between solid, dashed, and dotted lines
styles = [
    {"linestyle": "-", "lw": 4.0, 'alpha': 0.9},  # Solid line
    {"linestyle": "--", "lw": 4.0, 'alpha': 0.9},  # Dashed line
    {"linestyle": ":", "lw": 4.0, 'alpha': 0.9},  # Dotted line
    {"linestyle": "-.", "lw": 4.0, 'alpha': 0.9},  # Dash-dot line
]

# Plotting infection traces over time
keys = sorted(list(data_dict.keys()))
for (n, graph_type) in keys:
    data = data_dict[(n, graph_type)]

    plt.figure(figsize=(10, 6))

    # Create a color palette (shades of blue)
    palette = sns.color_palette("Blues", len(data))

    # Loop over each DataFrame in the list for (n, graph_type)
    for i, df in enumerate(data):
        style = styles[i % len(styles)]  # Cycle through the defined styles
        plt.plot(df['Time'], df['Infected_nodes'], color=palette[i], **style)  # Different shades of blue

    # Get the current axis and format the plot
    ax = plt.gca()
    format_plot(ax, f"Prevalence ({graph_type.capitalize()} - {n})", "Time", "Number of Infected Nodes")
    plt.xlim(xmin=-0.05)
    plt.ylim(ymin=-0.05)

    # Save the figure to a PDF file
    plt.savefig(f"Infection_traces_{graph_type}_{n}.pdf", bbox_inches='tight')

    # Show the plot
    plt.show()

# Plotting average degree variation over time
for (n, graph_type) in keys:
    data = data_dict[(n, graph_type)]

    plt.figure(figsize=(10, 6))

    # Create a color palette (shades of red)
    palette = sns.color_palette("Reds", len(data))

    # Loop over each DataFrame in the list for (n, graph_type)
    for i, df in enumerate(data):
        style = styles[i % len(styles)]  # Cycle through the defined styles
        plt.plot(df['Time'], df['Avg_degree'], color=palette[i], **style)  # Different shades of red

    # Get the current axis and format the plot
    ax = plt.gca()
    format_plot(ax, f"Degree Variation ({graph_type.capitalize()} - {n})", "Time", "Average Degree")
    plt.xlim(xmin=-0.1)

    # Save the figure to a PDF file
    plt.savefig(f"Avgdegree_traces_{graph_type}_{n}.pdf", bbox_inches='tight')

    # Show the plot
    plt.show()


### Plotting Results

In [ ]:
# Set seaborn style for publication-quality plots, without grid
sns.set(style="white", rc={"axes.edgecolor": "0.0", "axes.linewidth": 1.25})

# Function to format plots for publication
def format_plot(ax, title, xlabel, ylabel):
    """Format the plot for publication-ready figures."""
    ax.set_title(title, fontsize=30)
    ax.set_xlabel(xlabel, fontsize=30)
    ax.set_ylabel(ylabel, fontsize=30)
    ax.tick_params(axis='both', which='major', labelsize=30, length=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Initialize list to store data
data_list = []

# Get all files matching the pattern
files = glob.glob("sis_simulation_results_*.csv") + \
        glob.glob("sis_baselinefast_simulation_results_*.csv") + \
        glob.glob("sis_baselinenaive_simulation_results_*.csv")

# Process each file and store data in the list
for file in sorted(files):
    print(file)
    graph_type = file.split("_")[-1].replace('.csv', '')
    method = 'ours'  # Default method
    if 'fast' in file:
        method = 'baselinefast'
    elif 'naive' in file:
        method = 'baselinenaive'

    df = pd.read_csv(file)
    n = df['Number_of_nodes'].iloc[-1]
    runtime = df['Run_time_in_ms'].iloc[-1]
    accepted_events = df['Accepted_events'].iloc[-1]
    runtime_per_step = runtime / accepted_events if accepted_events > 0 else 0

    data_list.append((n, graph_type, method, runtime_per_step))

# Extract unique values for graph types, methods, and node numbers
graph_types = sorted(list(set([graph_type for n, graph_type, method, runtime_per_step in data_list])))
methods = sorted(list(set([method for n, graph_type, method, runtime_per_step in data_list])))
node_nums = sorted(list(set([n for n, graph_type, method, runtime_per_step in data_list])))

# Use seaborn color palettes for blue, red, and green shades
palette = sns.color_palette(["#d62728", "#1f77b4", "#2ca02c"])  # Red for ours, blue for baselinefast, green for baselinenaive

# Map methods to colors and markers
method_to_color = {
    'ours': palette[0],  # Red
    'baselinefast': palette[1],  # Blue
    'baselinenaive': palette[2]  # Green
}
method_to_marker = {
    'ours': '^',
    'baselinefast': '>',
    'baselinenaive': '<'
}

method_to_label = {
    'ours': 'icon (ours)',
    'baselinefast': 'Baseline (fast)',
    'baselinenaive': 'Baseline (naive)'
}

# List to store final results
final_results = []

# Plotting data for each graph type
for graph_type_i in graph_types:
    plt.close()
    plt.figure(figsize=(10, 6))
    for method_i in methods:
        for n_i in node_nums:
            # Filter data points for the current graph_type, method, and node number
            datapoints = [runtime_per_step for n, graph_type, method, runtime_per_step in data_list
                          if n == n_i and method == method_i and graph_type == graph_type_i]
            print("datapoints", datapoints)  # Debugging output
            if len(datapoints) == 0:
                continue
            runtime_mean = np.mean(datapoints)
            runtime_std = np.std(datapoints) if len(datapoints) > 1 else 1.0
            final = (graph_type_i, method_i, n_i, runtime_mean, runtime_std)
            final_results.append(final)

            # Plot scatter with different styles for 'ours' method
            plt.scatter([n_i], [runtime_mean], s=100, edgecolors=method_to_color[method_i],
                        facecolors='none', marker=method_to_marker[method_i], alpha=0.8,
                        linewidths=3 if method_i == 'ours' else 2)

    # Add dummy scatter plots for legend with hollow markers
    for method_i in methods:
        plt.scatter([], [], s=100, edgecolors=method_to_color[method_i], facecolors='none',
                    marker=method_to_marker[method_i], label=method_to_label[method_i], linewidths=2)

    # Set the x and y axis to logarithmic scale (log-log scale)
    plt.xscale('log')
    plt.yscale('log')

    # Format the plot
    plt.legend(title="Method", loc='center left', bbox_to_anchor=(1, 0.5), frameon=False,
               fontsize=22, title_fontsize=22)
    format_plot(plt.gca(), f"Runtime per Step for {graph_type_i.capitalize()} Graph",
                "Number of Nodes", "Runtime per Step (ms)")

    # Save the plot
    plt.savefig(f"Scatter_{graph_type_i}_loglog_hollow.pdf", bbox_inches='tight')
    plt.show()

# Print final results
print(final_results)
